## Introducción

El presente notebook tiene como objetivo principal la **extracción eficiente de estadísticas descriptivas clave** a partir de múltiples archivos CSV ubicados en una carpeta específica (`raw`). Este proceso se realiza mediante la biblioteca **Dask**, la cual permite trabajar de forma escalable y eficiente con conjuntos de datos de gran tamaño, incluso en entornos con recursos limitados.

El propósito final de este análisis es generar un **resumen estadístico detallado de cada archivo**, abarcando tanto variables numéricas como categóricas. Estos resúmenes son exportados en formato JSON, con el fin de **facilitar la posterior generación de dataframes sintéticos** que conserven las propiedades estadísticas esenciales de los datos originales, pero con una cantidad reducida de filas. Esta estrategia es particularmente útil en escenarios de validación, benchmarking, o simulación, donde se requiere preservar la estructura y comportamiento de los datos sin necesidad de trabajar con la totalidad de los registros originales.

La implementación hace uso de un enfoque *streaming y silencioso*, evitando sobrecargas en el sistema y maximizando la compatibilidad con entornos de ejecución restringidos (como máquinas con poca memoria RAM). Además, se implementa un control manual del scheduler de Dask para asegurar una ejecución secuencial predecible y controlada.


In [ ]:
import glob
import os
import contextlib
import json
import dask
import dask.dataframe as dd
import numpy as np
import pandas as pd
from pandas.api import types as pd_types

# 1) Desactivar cualquier progreso de Dask
dask.config.set({
    'progressbar': False,         # desactiva ProgressBar()
    'scheduler': 'synchronous'    # fuerza ejecución síncrona
})

# 2) Helper para computar “silencioso” y forzando el scheduler síncrono
def silent_compute(*args, **kwargs):
    kwargs.setdefault('scheduler', 'synchronous')
    with open(os.devnull, 'w') as fnull, \
         contextlib.redirect_stdout(fnull), \
         contextlib.redirect_stderr(fnull):
        return dask.compute(*args, **kwargs)


def compute_stats_streaming(path):
    # Leer CSV en Dask
    df = dd.read_csv(path, assume_missing=True, blocksize="100MB")

    # Obtener dtypes reales
    real_dtypes = df.dtypes
    numeric_cols = [col for col, dt in real_dtypes.items() if pd_types.is_numeric_dtype(dt)]
    cat_cols     = [col for col, dt in real_dtypes.items()
                    if pd_types.is_object_dtype(dt) or pd_types.is_categorical_dtype(dt) or pd_types.is_string_dtype(dt)]

    stats = {}

    # Estadísticas para columnas numéricas
    for col in numeric_cols:
        s = df[col]
        cnt, mn, sd, mnm, mx = silent_compute(
            s.count(), s.mean(), s.std(), s.min(), s.max()
        )
        # Percentiles con conversión segura a float
        try:
            q_vals = s.astype(float).quantile([0.25, 0.5, 0.75]) \
                          .compute(scheduler='synchronous')
            q25, q50, q75 = q_vals.values
        except Exception:
            q25 = q50 = q75 = None
        nan_cnt = silent_compute(s.isna().sum())[0]
        inf_cnt = silent_compute((s == np.inf).sum())[0]
        nin_cnt = silent_compute((s == -np.inf).sum())[0]

        stats[col] = {
            'count': int(cnt),
            'mean': float(mn),
            'std': float(sd),
            'min': float(mnm),
            '25%': float(q25) if q25 is not None else None,
            '50%': float(q50) if q50 is not None else None,
            '75%': float(q75) if q75 is not None else None,
            'max': float(mx),
            'NaN': int(nan_cnt),
            'Inf': int(inf_cnt),
            '-Inf': int(nin_cnt),
        }

    # Estadísticas para columnas categóricas
    for col in cat_cols:
        s = df[col]
        cnt  = silent_compute(s.count())[0]
        uniq = silent_compute(s.nunique())[0]
        vc = s.value_counts().nlargest(1).compute(scheduler='synchronous')
        if len(vc):
            top, freq = vc.index[0], int(vc.iloc[0])
        else:
            top, freq = None, 0
        nan_cnt = silent_compute(s.isna().sum())[0]

        # Intentar longitud de cadenas
        try:
            lengths = df[col].str.len()
            mn_len, mx_len = silent_compute(lengths.mean(), lengths.max())
            stats[col] = {
                'count': int(cnt),
                'unique': int(uniq),
                'top': top,
                'freq': freq,
                'NaN': int(nan_cnt),
                'mean_length': float(mn_len),
                'max_length': int(mx_len)
            }
        except Exception:
            stats[col] = {
                'count': int(cnt),
                'unique': int(uniq),
                'top': top,
                'freq': freq,
                'NaN': int(nan_cnt)
            }

    return stats


if __name__ == '__main__':
    # Carpeta con los CSV
    folder    = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw"
    csv_files = glob.glob(os.path.join(folder, "*.csv"))

    all_stats = {}
    for path in csv_files:
        fname = os.path.basename(path)
        print(f"Procesando {fname}...")
        stats = compute_stats_streaming(path)
        all_stats[fname] = stats

    # Guardar resultados en JSON
    output_path = os.path.join(folder, 'all_stats.json')
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(all_stats, f, ensure_ascii=False, indent=4)
    print(f"Todos los resultados se han guardado en: {output_path}")



Procesando holidays_events.csv...
[#####                                   ] | 14% Completed | 107.27 ms

C:\Users\spinz\AppData\Local\Temp\ipykernel_28628\2013412933.py:34: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd_types.is_object_dtype(dt) or pd_types.is_categorical_dtype(dt) or pd_types.is_string_dtype(dt)]


[########################################] | 100% Completed | 615.21 ms
[########################################] | 100% Completed | 629.41 ms
[########################################] | 100% Completed | 634.41 ms
[########################################] | 100% Completed | 730.01 ms
[########################################] | 100% Completed | 730.95 ms
[########################################] | 100% Completed | 823.89 ms
[########################################] | 100% Completed | 918.90 ms
[########################################] | 100% Completed | 1.02 s
[########################################] | 100% Completed | 1.03 s
[########################################] | 100% Completed | 1.12 s
[########################################] | 100% Completed | 1.12 s
[########################################] | 100% Completed | 1.22 s
[########################################] | 100% Completed | 1.22 s
[########################################] | 100% Completed | 1.32 s
[############

C:\Users\spinz\AppData\Local\Temp\ipykernel_28628\2013412933.py:34: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd_types.is_object_dtype(dt) or pd_types.is_categorical_dtype(dt) or pd_types.is_string_dtype(dt)]


[########################################] | 100% Completed | 943.52 ms
[########################################] | 100% Completed | 977.86 ms
[########################################] | 100% Completed | 1.01 s
[########################################] | 100% Completed | 1.11 s
[########################################] | 100% Completed | 1.18 s
[########################################] | 100% Completed | 1.27 s
[########################################] | 100% Completed | 1.30 s
[########################################] | 100% Completed | 1.39 s
[########################################] | 100% Completed | 1.40 s
[########################################] | 100% Completed | 1.50 s
[########################################] | 100% Completed | 1.57 s
[########################################] | 100% Completed | 1.67 s
[########################################] | 100% Completed | 1.69 s
[########################################] | 100% Completed | 1.69 s
[###########################

...

[########################################] | 100% Completed | 174.71 s
[########################################] | 100% Completed | 174.74 s
[########################################] | 100% Completed | 174.84 s
[########################################] | 100% Completed | 174.89 s
[########################################] | 100% Completed | 174.95 s
[########################################] | 100% Completed | 174.98 s
[########################################] | 100% Completed | 175.08 s
[########################################] | 100% Completed | 175.13 s
[########################################] | 100% Completed | 175.18 s
[########################################] | 100% Completed | 175.20 s
[########################################] | 100% Completed | 175.27 s
[########################################] | 100% Completed | 175.29 s
[########################################] | 100% Completed | 175.32 s
[########################################] | 100% Completed | 175.34 s
[#####

c:\Users\spinz\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[########################################] | 100% Completed | 129.94 s
[########################################] | 100% Completed | 129.96 s
[########################################] | 100% Completed | 130.05 s
[########################################] | 100% Completed | 130.07 s
[########################################] | 100% Completed | 130.08 s
[########################################] | 100% Completed | 130.16 s
[########################################] | 100% Completed | 130.24 s
[########################################] | 100% Completed | 130.25 s
[########################################] | 100% Completed | 130.33 s
[########################################] | 100% Completed | 130.35 s
[########################################] | 100% Completed | 130.37 s
[########################################] | 100% Completed | 130.42 s
[########################################] | 100% Completed | 130.52 s
[########################################] | 100% Completed | 130.59 s
[#####

C:\Users\spinz\AppData\Local\Temp\ipykernel_28628\2013412933.py:34: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd_types.is_object_dtype(dt) or pd_types.is_categorical_dtype(dt) or pd_types.is_string_dtype(dt)]


[########################################] | 100% Completed | 739.44 ms
[########################################] | 100% Completed | 739.71 ms
[########################################] | 100% Completed | 838.38 ms
[########################################] | 100% Completed | 838.60 ms
[########################################] | 100% Completed | 935.25 ms
[########################################] | 100% Completed | 1.03 s
[########################################] | 100% Completed | 1.05 s
[########################################] | 100% Completed | 1.05 s
[########################################] | 100% Completed | 1.15 s
[########################################] | 100% Completed | 1.15 s
[########################################] | 100% Completed | 1.15 s
[########################################] | 100% Completed | 1.13 s
[########################################] | 100% Completed | 1.13 s
[########################################] | 100% Completed | 1.13 s
[##################